# Анализ тональности отзывов

Сначала возьмем выборку отзывов на фильмы из NLTK:

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import nltk
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/artem/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [3]:
from nltk.corpus import movie_reviews
 
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

print(negids[:5])

['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt']


Приготовим список текстов и классов как обучающую выборку:

In [4]:
negfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in negids]
posfeats = [" ".join(movie_reviews.words(fileids=[f])) for f in posids]

texts = negfeats + posfeats
labels = [0] * len(negfeats) + [1] * len(posfeats)

In [5]:
print(texts[1])

the happy bastard ' s quick movie review damn that y2k bug . it ' s got a head start in this movie starring jamie lee curtis and another baldwin brother ( william this time ) in a story regarding a crew of a tugboat that comes across a deserted russian tech ship that has a strangeness to it when they kick the power back on . little do they know the power within . . . going for the gore and bringing on a few action sequences here and there , virus still feels very empty , like a movie going for all flash and no substance . we don ' t know why the crew was really out in the middle of nowhere , we don ' t know the origin of what took over the ship ( just that a big pink flashy thing hit the mir ) , and , of course , we don ' t know why donald sutherland is stumbling around drunkenly throughout . here , it ' s just " hey , let ' s chase these people around with some robots " . the acting is below average , even from the likes of curtis . you ' re more likely to get a kick out of her work i

Импортируем нужные нам модули

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

### Оценка качества работы разных классификаторов

In [7]:
def text_classifier(vectorizer, transformer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("transformer", transformer),
            ("classifier", classifier)]
        )

In [8]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(clf)
    cv_mean = cross_val_score(
        text_classifier(CountVectorizer(),
                        TfidfTransformer(),
                        clf()),
        texts,
        labels,
        cv=5).mean()
    print(cv_mean)
    print("\n")

<class 'sklearn.linear_model.logistic.LogisticRegression'>
0.8210000000000001


<class 'sklearn.svm.classes.LinearSVC'>
0.8545


<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>
0.8390000000000001




### Подготовка классификатора, обученного на всех данных

In [9]:
clf_pipeline = Pipeline(
            [("vectorizer", TfidfVectorizer()),
            ("classifier", LinearSVC())]
        )


clf_pipeline.fit(texts, labels)

print(clf_pipeline)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])


In [10]:
print(clf_pipeline.predict(["Amazing film! I will advice it to all my friends. Genious",
                           "Awful film! The man who advised me to watch it is really crazy idiot."]))

[1 0]


## Понижение размерности и ансамбли деревьев

In [11]:
%%time
from sklearn.decomposition import NMF, TruncatedSVD

v = CountVectorizer()
mx = v.fit_transform(texts)
mf = TruncatedSVD(10)
u = mf.fit_transform(mx)

CPU times: user 2.17 s, sys: 122 ms, total: 2.29 s
Wall time: 1.33 s


In [12]:
for transform in [TruncatedSVD, NMF]:
    print(transform)
    cv_score = cross_val_score(
        text_classifier(
            CountVectorizer(),
            transform(n_components=10),
            LinearSVC()),
        texts, labels, cv=5).mean()
    
    print(cv_score)
    print("\n")


<class 'sklearn.decomposition.truncated_svd.TruncatedSVD'>
0.541


<class 'sklearn.decomposition.nmf.NMF'>
0.655







Если задать n_components=1000:

In [13]:
%%time
print(cross_val_score(
    text_classifier(
        TfidfVectorizer(), 
        TruncatedSVD(n_components=1000),
        LinearSVC()),
    texts, 
    labels,
    cv=5
    ).mean())

0.852
CPU times: user 3min 15s, sys: 21 s, total: 3min 36s
Wall time: 1min 43s


## Ансамбли деревьев на преобразованных признаках

In [14]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [15]:
%%time
print(cross_val_score(
    Pipeline([
            ("vectorizer", CountVectorizer()),
            ("transformer", TruncatedSVD(100)),
            ("classifier", RandomForestClassifier(100))
        ]),
    texts,
    labels,
    cv=5
    ))

[0.7075 0.675  0.7275 0.7475 0.72  ]
CPU times: user 29.6 s, sys: 1.52 s, total: 31.1 s
Wall time: 14.4 s


##### Больше компонент и больше деревьев:

In [18]:
%%time
print(cross_val_score(
    text_classifier(
        CountVectorizer(),
        TruncatedSVD(n_components=1000),
        RandomForestClassifier(1000)),
    texts, 
    labels
    ).mean())

0.716520412628197
CPU times: user 2min 47s, sys: 11.2 s, total: 2min 58s
Wall time: 2min


##### Tf*Idf вместо частот слов:

In [19]:
%%time
print(cross_val_score(
    text_classifier(
        TfidfVectorizer(),
        TruncatedSVD(n_components=1000),
        RandomForestClassifier(1000)),
    texts, 
    labels
    ).mean())

0.5905081728435021
CPU times: user 2min 47s, sys: 10.8 s, total: 2min 58s
Wall time: 1min 58s


##### Совмещаем Tf*Idf и SVD

In [20]:
from sklearn.pipeline import FeatureUnion

estimators = [('tfidf', TfidfTransformer()),
              ('svd', TruncatedSVD(1))]
combined = FeatureUnion(estimators)

In [21]:
%%time
print(cross_val_score(
    Pipeline([
            ("vectorizer", CountVectorizer()),
            ("transformer", combined),
            ("classifier", LinearSVC())
        ]),
    texts,
    labels
    ))

[0.6497006  0.74174174 0.62162162]
CPU times: user 9.76 s, sys: 185 ms, total: 9.95 s
Wall time: 7.1 s
